In [ ]:
#| default_exp models.tsmixer

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# TSMixer

Time-Series Mixer (`TSMixer`), a novel architecture designed by stacking multi-layer perceptrons (`MLPs`) that focuses on mixing time and feature dimensions to make better predictions.It is designed for time series forecasting, specifically in scenarios where the data is multivariate and exhibits complex dynamics. 

(`TSMixer`) presents a promising approach to time series forecasting by effectively leveraging both temporal and cross-variate information through a simple yet powerful architecture.

(`TSMixer`) demonstrates superior performance compared to the state-of-the-art alternatives on the challenging and large scale M5 benchmark

**References**<br>
-[Si-An Chen, Chun-Liang Li, Nathanael C. Yoder, Sercan Ö. Arık,Tomas Pfister. "TSMixer: An All-MLP Architecture for Time Series Forecasting".](https://arxiv.org/pdf/2303.06053.pdf)

![Figure 1. TSMixer Model Architecture.](imgs_models/tsmixer.png) 

In [ ]:
#| hide
from fastcore.test import test_eq
from nbdev.showdoc import show_doc
import matplotlib.pyplot as plt

In [ ]:
#| export
import neuralforecast
from typing import Optional

import torch
import torch.nn as nn
import neuralforecast

from neuralforecast.losses.pytorch import MAE
from neuralforecast.common._base_multivariate import BaseMultivariate

In [ ]:
#| export

class TransposeLayer(nn.Module):
    def __init__(self, dim0, dim1):
        super(TransposeLayer, self).__init__()
        self.dim0 = dim0
        self.dim1 = dim1

    def forward(self, x):
        return torch.transpose(x, self.dim0, self.dim1)

class ResBlock(nn.Module):
    def __init__(self, inputs, norm_type='L', activation=nn.ReLU, dropout=0., ff_dim=None):
        super(ResBlock, self).__init__()
        
        # inputs  - # [Batch, Input Length, Channel] [0,1,2]
        
        if norm_type == 'L':
            self.norm = nn.LayerNorm
        else:
            self.norm = nn.BatchNorm1d


        # print(inputs[-1])
        # print(ff_dim)
        
        # Temporal Linear
        self.temporal_linear = nn.Sequential(
            self.norm(1),         
            TransposeLayer(1, 2), # [Batch, Channel, Input Length] 
            nn.Linear(inputs[-1], inputs[-1], bias=False),
            activation(), 
            TransposeLayer(1, 2),
            nn.Dropout(dropout)
            
        )

        # Feature Linear
        self.feature_linear = nn.Sequential(
            self.norm(1),
            nn.Linear(inputs[-1],ff_dim, bias=False),
            activation(),
            nn.Dropout(dropout),
            nn.Linear(ff_dim, inputs[-1], bias=False),
            activation(),
            nn.Dropout(dropout)
        )
        
    def forward(self, inputs):
        # Temporal Linear
        x = self.temporal_linear(inputs)
        res = x + inputs

        # Feature Linear
        x = self.feature_linear(res)
        return x + res

In [ ]:
#| export
class TSMixer(BaseMultivariate):
    """ TSMixer
    
    Basic TSMixer Model architecture.    
    This deep neural network model builds upon the idea that linear models are effective for capturing time dependencies. 
    The TSMixer architecture combines linear models with non-linearities, stacking them in alternating layers for time and feature domains. 
    This approach leverages both temporal patterns and covariate information. 
    The model includes time-mixing and feature-mixing MLPs, temporal projection, residual connections, and 2D normalization.
    It is trained using ADAM stochastic gradient descent.
    The network accepts static, historic and future exogenous data, flattens the inputs and learns fully connected relationships against the target variable.
    
    **Parameters:**<br> 
    `h`: int, forecast horizon.<br>
    `input_size`: int, considered autorregresive inputs (lags), y=[1,2,3,4] input_size=2 -> lags=[1,2].<br> 
    `n_series`: int, number of time-series.<br>
    `loss`: PyTorch module, instantiated train loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
    `valid_loss`: PyTorch module=`loss`, instantiated valid loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
    `learning_rate`: float=1e-3, Learning rate between (0, 1).<br>
    `max_steps`: int=1000, maximum number of training steps.<br>
    `val_check_steps`: int=100, Number of training steps between every validation loss check.<br>
    `n_series`:int, the number of time series in the multivariate time series data. <br>
    `batch_size`: int=32, number of different series in each batch.<br>
    `step_size`: int=1, step size between each window of temporal data.<br>
    `num_lr_decays`: int=-1, Number of learning rate decays, evenly distributed across max_steps.<br>
    `early_stop_patience_steps`: int=-1, Number of validation iterations before early stopping.<br>
    `hidden_size`: int, number of units for each layer of the MLP.<br>
    `n_layers`: int, number of layers for the MLP.<br>
    `scaler_type`: str='identity', type of scaler for temporal inputs normalization see [temporal scalers](https://nixtla.github.io/neuralforecast/common.scalers.html).<br>
    `futr_exog_list`: str list, future exogenous columns.<br>
    `hist_exog_list`: str list, historic exogenous columns.<br>
    `stat_exog_list`: str list, static exogenous columns.<br>
    `scaler_type`: str='identity', type of scaler for temporal inputs normalization see [temporal scalers](https://nixtla.github.io/neuralforecast/common.scalers.html).<br>
    `num_workers_loader`: int=os.cpu_count(), workers to be used by `TimeSeriesDataLoader`.<br>
    `drop_last_loader`: bool=False, if True `TimeSeriesDataLoader` drops last non-full batch.<br>
    `random_seed`: int=1, random_seed for pytorch initializer and numpy generators.<br>
    `alias`: str, optional, Custom name of the model.<br>    
    `**trainer_kwargs`: int, keyword trainer arguments inherited from [PyTorch Lighning's trainer](https://pytorch-lightning.readthedocs.io/en/stable/api/pytorch_lightning.trainer.trainer.Trainer.html?highlight=trainer).<br>    
    
    `Additional parameters for TSMixer model:-  
    `activation`: str, activation from ['ReLU', 'Softplus', 'Tanh', 'SELU', 'LeakyReLU', 'PReLU', 'Sigmoid'].<br> 
    `n_block`: int , number of mixer layers/residual blocks.<br>
    `dropout`: float=0., dropout regularization.<br>
    `ff_dim`:int, number of features or characteristics the model should consider when processing information about the data. <br>    
    `target_slice`: list[start,end] the target_slice parameter is used to customize the predicted sequence the model should focus on. When target_slice=None - In this case, the model predicts the entire sequence.<br>   
    """
    
    # Class attributes
    SAMPLING_TYPE = 'windows'
    
    def __init__(self,
                 h,                          # prediction length                 
                 input_size,                 # Autoregressive terms
                 loss=MAE(),
                 valid_loss=None,
                 learning_rate: float = 1e-3,
                 max_steps: int = 1000,
                 val_check_steps: int = 100,
                 step_size: int = 1,
                 num_lr_decays: int = 0,
                 early_stop_patience_steps: int =-1,
                 # hidden_size: int = 1,
                 # n_layers: int = 1,
                 scaler_type: str = 'robust',
                 futr_exog_list=None,
                 hist_exog_list=None,
                 stat_exog_list=None,
                 num_workers_loader: int = 0,
                 drop_last_loader: bool = False,
                 random_seed: int =1, 
                 alias=None,
                 
                 ###### Specific to TSMixer                 
                 n_series:int =None,        # Number of input channels
                 batch_size: int = 32,      # The input batch size
                 # activation: str = 'ReLU',  # Activation required to normalize the MLP 
                 n_block: int =None,        # Number of residual blocks
                 dropout:float =.01,        # Droput for MLP layers
                 ff_dim:int = None,         # Feature dimensions for each layers
                 target_slice:list =None,   # Prediction window user wnats to analyze
                 ######
                 **trainer_kwargs
                ):
        
  
        super(TSMixer, self).__init__(
                                      h=h,
                                      input_size=input_size,
                                      loss=loss,
                                      valid_loss=valid_loss,
                                      learning_rate=learning_rate,
                                      max_steps=max_steps,
                                      val_check_steps=val_check_steps,                                      
                                      step_size=step_size,
                                      num_lr_decays=num_lr_decays,
                                      early_stop_patience_steps=early_stop_patience_steps,
                                      # hidden_size=hidden_size,
                                      # n_layers=n_layers,
                                      scaler_type=scaler_type,
                                      futr_exog_list=futr_exog_list,
                                      hist_exog_list=hist_exog_list,
                                      stat_exog_list=stat_exog_list,
                                      num_workers_loader=num_workers_loader,
                                      drop_last_loader=drop_last_loader,
                                      random_seed=random_seed, 
                                      alias=alias,
            
                                      ##### Specific to TSMixer                                       
                                      # activation = activation,
                                      n_block=n_block,
                                      dropout=dropout,
                                      ff_dim=ff_dim,
                                      target_slice=target_slice,
                                      n_series=n_series,
                                      batch_size=batch_size,
                                      #####
                                      **trainer_kwargs
                                     )
        

        
        # self.h = h
        # self.input_size = input_size
        self.n_series = n_series

        #Parameters specific to TSMixer.
        # self.activation = activation
        self.n_block = n_block
        self.dropout = dropout
        self.ff_dim = ff_dim
        self.target_slice = target_slice
        self.batch_size = batch_size
        
        
        
        # Create TSMixer-specific modules with learnable parameters        
        input_shape = (batch_size, input_size, n_series)  # Assuming batch_size, input_size, and n_series are user inputs
   
        self.res_blocks = nn.ModuleList([ResBlock(inputs = input_shape, dropout=dropout, ff_dim=ff_dim) for _ in range(self.n_block)])

        self.output_layer = nn.Sequential(
            TransposeLayer(1, 2),
            nn.Linear(input_shape[-1],h),
            TransposeLayer(1, 2),
        )
        
    def forward(self, windows_batch):
        
        x = windows_batch['insample_y']
        # insample_mask = windows_batch['insample_mask']
        # futr_exog = windows_batch['futr_exog']
        # hist_exog = windows_batch['hist_exog']
        # stat_exog = windows_batch['stat_exog']
        
        input_shape = x.shape[1:]
        batch_size = x.size(0)  # Get the dynamic batch size

        for res_block in self.res_blocks:
            x = res_block(x.view(batch_size,*input_shape))  # Reshape to include batch_size

        if self.target_slice:
            x = x[:, :, self.target_slice]

        x = self.output_layer(x)

        y_pred = x.reshape(x.size(0), self.h, self.loss.outputsize_multiplier)
        # y_pred = x.view(batch_size, self.h, self.loss.outputsize_multiplier)
        y_pred = self.loss.domain_map(y_pred)

        return y_pred